In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from xgboost import XGBClassifier
from sklearn import neighbors
from sklearn import metrics
from sklearn import model_selection

In [3]:
baseDir = r'.\Data'
li = []
for fileName in os.listdir(baseDir):
    currentFilePath = os.path.join(baseDir, fileName)
    df1 = pd.read_csv(currentFilePath, header=None)
    df1[0] = fileName
    li.append(df1)
df = pd.concat(li)
df[0].unique()

array(['1.csv', '10.csv', '11.csv', '12.csv', '13.csv', '14.csv',
       '15.csv', '2.csv', '3.csv', '4.csv', '5.csv', '6.csv', '7.csv',
       '8.csv', '9.csv'], dtype=object)

In [4]:
df.rename(columns = {0:'filename', 1:'x_acc(pitch)', 2:'y_acc(roll)', 3:'z_acc(yaw)', 4:'label'}, inplace = True)

In [5]:
df.head()

,filename,x_acc(pitch),y_acc(roll),z_acc(yaw),label
0,1.csv,1502,2215,2153,1
1,1.csv,1667,2072,2047,1
2,1.csv,1611,1957,1906,1
3,1.csv,1601,1939,1831,1
4,1.csv,1643,1965,1879,1


In [6]:
df.label.value_counts()

1    608667
7    593563
4    357064
3    216737
5     51498
2     47878
6     47770
0      3719
Name: label, dtype: int64

In [7]:
df = df[df['label'] != 0]
df = df[df['label'] != 6]
df = df[df['label'] != 2]
df = df[df['label'] != 5]
df.label.value_counts()

1    608667
7    593563
4    357064
3    216737
Name: label, dtype: int64

In [8]:
df.head()

,filename,x_acc(pitch),y_acc(roll),z_acc(yaw),label
0,1.csv,1502,2215,2153,1
1,1.csv,1667,2072,2047,1
2,1.csv,1611,1957,1906,1
3,1.csv,1601,1939,1831,1
4,1.csv,1643,1965,1879,1


In [9]:
Q1 = df['x_acc(pitch)'].quantile(0.25)
Q3 = df['x_acc(pitch)'].quantile(0.75)
IQR = Q3 - Q1    #IQR is interquartile range. 
filter = (df['x_acc(pitch)'] >= Q1 - 1.5 * IQR) & (df['x_acc(pitch)'] <= Q3 + 1.5 *IQR)
df1 = df.loc[filter] 

In [10]:
Q1 = df['y_acc(roll)'].quantile(0.25)
Q3 = df['y_acc(roll)'].quantile(0.75)
IQR = Q3 - Q1    #IQR is interquartile range. 
filter = (df['y_acc(roll)'] >= Q1 - 1.5 * IQR) & (df['y_acc(roll)'] <= Q3 + 1.5 *IQR)
df1 = df.loc[filter] 

In [11]:
Q1 = df['z_acc(yaw)'].quantile(0.25)
Q3 = df['z_acc(yaw)'].quantile(0.75)
IQR = Q3 - Q1    #IQR is interquartile range. 
filter = (df['z_acc(yaw)'] >= Q1 - 1.5 * IQR) & (df['z_acc(yaw)'] <= Q3 + 1.5 *IQR)
df1 = df.loc[filter] 

In [12]:
print(df1.shape)
print(df.shape)

(1724450, 5)
(1776031, 5)


In [13]:
df1["magnitude"]=(df1['x_acc(pitch)']**2+df1['y_acc(roll)']**2+df1['z_acc(yaw)']**2)**(1/2)
df1["mean"]=(df1['x_acc(pitch)']+df1['y_acc(roll)']+df1['z_acc(yaw)'])/3

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
df1['magnitude'] = df1.magnitude.astype(int)
df1['mean']=df1['mean'].astype(int)
df1.head()

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,filename,x_acc(pitch),y_acc(roll),z_acc(yaw),label,magnitude,mean
0,1.csv,1502,2215,2153,1,3434,1956
1,1.csv,1667,2072,2047,1,3355,1928
2,1.csv,1611,1957,1906,1,3171,1824
3,1.csv,1601,1939,1831,1,3110,1790
4,1.csv,1643,1965,1879,1,3176,1829


In [16]:
def modelstats1(Xtrain,Xtest,ytrain,ytest):
    stats=[]
    modelnames = ["KNN"]
    models=list()
    models.append(neighbors.KNeighborsClassifier())
    for name,model in zip(modelnames,models):
        if name=="KNN":
            for k in range(29,30):
                model = neighbors.KNeighborsClassifier(n_neighbors=k, p=2)
                model.fit(Xtrain,ytrain)
                trainprediction=model.predict(Xtrain)
                testprediction=model.predict(Xtest)
                scores=list()
                scores.append(name+"-train")
                scores.append(metrics.accuracy_score(ytrain,trainprediction))
                stats.append(scores)
                scores=list()
                scores.append(name+"-test")
                scores.append(metrics.accuracy_score(ytest,testprediction))
                stats.append(scores)
                print("for k = ",k,"training accuracy = ",metrics.accuracy_score(ytrain,trainprediction),"testing accuracy = ",metrics.accuracy_score(ytest,testprediction))
        else:
            model.fit(Xtrain,ytrain)
            
        colnames=["MODELNAME","ACCURACY"]
    return pd.DataFrame(stats,columns=colnames)

In [22]:
X = df1.drop(['filename','label'], axis=1)
y = df1['label']
Xtrain,Xtest,ytrain,ytest=model_selection.train_test_split(X,y,test_size=.2,random_state=42)
modelstats1(Xtrain,Xtest,ytrain,ytest)

for k =  29 training accuracy =  0.8133412102409464 testing accuracy =  0.8051233726695468


,MODELNAME,ACCURACY
0,KNN-train,0.813341
1,KNN-test,0.805123


In [19]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Xtrain,Xtest,ytrain,ytest=model_selection.train_test_split(X,y,test_size=.2,random_state=42)
# fit model no training data
model = XGBClassifier()
model.fit(Xtrain, ytrain)

C:\Users\HP\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:07:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=100, n_jobs=4, num_parallel_tree=1,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=None, subsample=1,
       tree_method='exact', use_label_encoder=True, validate_parameters=1,
       verbosity=None)

In [21]:
y_pred = model.predict(Xtest)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(ytest, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 80.04%
